In [ ]:
import requests
import pandas as pd
import datetime
import time

# Carica i dati dal CSV
file_path = 'data/HistoricalData_TSLA.csv'
data = pd.read_csv(file_path)

# Funzione per convertire la data in timestamp epoch
def date_to_epoch(date_str, hour=0, minute=0, second=0):
    dt = datetime.datetime.strptime(date_str, "%m/%d/%Y")
    dt = dt.replace(hour=hour, minute=minute, second=second)
    return int(dt.timestamp())

# Funzione per inviare i dati all'API
def send_data_to_api(date, value, ticker='TSLA', data_type='open'):
    url = "http://localhost:8080/DatabaseManager/stock-api"
    headers = {'Content-Type': 'application/json'}
    numeric_value = float(value.replace('$', ''))
    # numeric_value = value
    payload = {
        "ticker": ticker,
        "price": numeric_value,
        "timestamp": date_to_epoch(date, hour={'open': 9, 'low': 10, 'high': 11, 'close': 16}[data_type])
    }
    print(f"Sending data to API: {payload}")
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 201:
        print(f"Data for {data_type} on {date} saved successfully.")
    else:
        print(f"Failed to save data for {data_type} on {date}. Response: {response.text}")

# Itera attraverso il DataFrame e invia i dati
for index, row in data.iterrows():
    date = row['Date']
    send_data_to_api(date, row['Open'], data_type='open')
    # time.sleep(1)
    send_data_to_api(date, row['Low'], data_type='low')
    # time.sleep(1)
    send_data_to_api(date, row['High'], data_type='high')
    # time.sleep(1)
    send_data_to_api(date, row['Close/Last'], data_type='close')
    # time.sleep(1)
